# Casimir Athena

Sandbox for playing with Casimir data in [AWS Athena](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/athena.html)

Configure AWS credentials and get last 10 events from Athena

In [7]:
import os
from pyathena import connect
import boto3
import boto3.session

session = boto3.Session(profile_name='consensus-networks-dev')
creds = session.get_credentials()

cursor = connect(
    aws_access_key_id=creds.access_key, 
    aws_secret_access_key=creds.secret_key, 
    s3_staging_dir="s3://casimir-etl-output-bucket-dev/", 
    region_name="us-east-2"
).cursor()

cursor.execute("SELECT * FROM casimir_etl_database_dev.casimir_etl_event_table_dev LIMIT 10")

for row in cursor:
    print(row)

('create_stake', datetime.date(2021, 8, 29), '04776260f30db5e44a1072f3904a6377d2278459e6b55caa815b95bc588c566cc71a0f12007b1f08adc09239ecb7c2683e9c8d9dff85745f873d970af1eef11510', 'iotexteam', '673000000000000000000', 91, False)
('create_stake', datetime.date(2021, 8, 29), '049b59c1f0347af0da32c76b749effebadb2d4bcde32b71d1f0bd49da1453b001b4ff5cebc56cf8e984a6ba1bb544fca6813f562a7077e9c83ae34394633509a93', 'binancevote', '100000000000000000000', 35, False)
('create_stake', datetime.date(2021, 8, 29), '047d90d205b8d23271c2f9ee1bffa88ec76e6b981adeace49b647db685734273773965af4f3b539cb3c2fada7a8799117b2776bdfb54c6ce05884af8ce88e50054', 'emmasiotx', '100000000000000000000', 100, True)
('create_stake', datetime.date(2021, 8, 28), '0439f264a2f9692d813bdb600816508542019c176537ac06791d74fb5ea8bdf04933c5ae22254f3bd9e7042aee03eb9aa6498a8c8e766bd65e0f40d65b42e14e42', 'iotfi', '900000000000000000000', 14, False)
('create_stake', datetime.date(2021, 8, 28), '04f1d1d299a713586e60ac48792729806b41a381e7e5